# Extrair grupos de textos dos acórdãos e pareceres prévios
Versão 2

## Introdução
Cria dataframe a partir de **tce_acordaos_e_pareceres.csv**. Esse dataset já possui os campos:
- tipo_processo
- cod_ato
- tipo_ato
- Peca
- Caminho
- conselheiro
- Texto
- Linhas
- decisao_tipo
- decisao_numero
- decisao_ano
- cod_processo
- cod_processo_aj

A partir do campo Linhas, serão extraídos os seguintes grupos de textos presentes nos acórdãos:
* Informações processuais: grupo que contém número do processo, assunto, relator, produrador, etc...
* Ementa
* Sumário
* Disposições: Contendo todo o restante do acórdão. Pode ser importante fazer sua divisão depois

Após extração, remove algumas ementas incorretas<br/>
Esporta dados para arquivo 'data/raw/tce_acordaos_e_pareceres_com_ementas.csv' para uso posterior

OBS:
- Grupos foram separados com base na sequência de aparecimento esperada no texto
- Para bom funcionamento, a ementa precisa estar recuada, como impõe [Resuloução do TCE nº 15 de 14/08/2017](https://www.tce.pi.gov.br/wp-content/uploads/2017/08/Resolu%C3%A7%C3%A3o-n%C2%BA-15-17-Elabora%C3%A7%C3%A3o-de-ac%C3%B3rd%C3%A3os-e-ementas.pdf)
- Pode ser interessante extrair dados de informações processuais em segundo momento
- Para poder relacionar com outros datasets, devem ser guardados os campos cod_tce, cod_ato e peça

Criado por Antonio Filho em 08/09/2020 (v1) e alterado em 03/10/2020 (v2 - esta)


## Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

import os
import sys
sys.path.insert(0, os.path.abspath('../../src'))
#sys.path.insert(0, os.path.abspath('../../src/preparation'))

from acordaos_utils import clean_text

## Criar dataset de todas as decisões (acórdãos e pareceres)

In [18]:
caminho_dados = Path("../../data/raw/")
df_decisoes = pd.read_csv(caminho_dados/"tce_acordaos_e_pareceres.csv", sep=";")

In [19]:
df_decisoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18921 entries, 0 to 18920
Data columns (total 20 columns):
cod_tce            18921 non-null object
assunto            17073 non-null object
tipo_processo      18921 non-null object
cod_ato            18921 non-null int64
ato                18921 non-null object
tipo_ato           18921 non-null object
peca               18921 non-null int64
caminho            18921 non-null object
conselheiro        15713 non-null object
municipio          15792 non-null object
ug                 15792 non-null object
procurador         15639 non-null object
ano_exercicio      15750 non-null float64
texto              18877 non-null object
linhas             18918 non-null object
decisao_tipo       18866 non-null object
decisao_numero     18866 non-null object
decisao_ano        18866 non-null float64
cod_processo       18921 non-null object
cod_processo_aj    18921 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 2.9+ MB


In [4]:
df_decisoes.head()

,cod_tce,assunto,tipo_processo,cod_ato,ato,tipo_ato,peca,caminho,conselheiro,municipio,ug,procurador,ano_exercicio,texto,linhas,decisao_tipo,decisao_numero,decisao_ano,cod_processo,cod_processo_aj
0,000132/2017,DENÚNCIA REF. IRREGULARIDADES NA ADMINISTRAÇÃO...,CONTROLE SOCIAL - DENÚNCIA,1284960,ACOCAM - G.LNM - 684/2017,ACÓRDÃO DE CÂMARA,23,/2017/06/688336/133/2/805temp/ACOCAM.pdf,NaN,NaN,NaN,NaN,NaN,ACORDÃO Nº 1647/2017\n\n\nPROCESSO TC Nº 00013...,"[{'page': 1, 'x0': 245, 'x1': 380, 'bottom': 1...",acordao,1647,2017.0,TC/000132/2017,TC/000132/2017
1,000189/2014,Aposentadoria,INATIVAÇÃO - APOSENTADORIA,262333,ACOCAM - GLUCIANO - 65/2014,ACÓRDÃO DE CÂMARA,10,/2014/07/623635/12/2/587temp/ACOCAM.pdf,NaN,NaN,NaN,NaN,NaN,ACÓRDÃO N.º 788/2014\n\nProcesso TC/000189/201...,"[{'page': 1, 'x0': 70, 'x1': 316, 'bottom': 13...",acordao,788,2014.0,TC/000189/2014,TC/000189/2014
2,000135/2020,RECURSO DE RECONSIDERAÇÃO REF. AO TC/005130/2015,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,2462530,ACOCAM - G.WA - 690/2020,ACÓRDÃO DE CÂMARA,15,/2020/03//770683/8/2/17032020_095623_46033325D...,NaN,NaN,NaN,NaN,NaN,ACÓRDÃO Nº 317/2020\nPROCESSO: TC/000135/2020\...,"[{'page': 1, 'x0': 245, 'x1': 377, 'bottom': 1...",acordao,317,2020.0,TC/000135/2020,TC/000135/2020
3,000271/2017,AUDITORIA CONCOMITANTE - EXERCICIO DE 2016,FISCALIZAÇÃO - AUDITORIA,1389052,ACOPLE - G.LNM - 281/2017,ACÓRDÃO DO PLENO,35,/2017/09/688475/133/3/604temp/ACOPLE.pdf,NaN,NaN,NaN,NaN,NaN,ACORDÃO Nº 2.504/17\n\nPROCESSO TC Nº 000271/2...,"[{'page': 1, 'x0': 250, 'x1': 376, 'bottom': 1...",acordao,2504,2017.0,TC/000271/2017,TC/000271/2017
4,000244/2017,"DENÚNCIA REF. IRREGULARIDADES NA FUESPI, EXERC...",CONTROLE SOCIAL - DENÚNCIA,1255656,ACOCAM - G.LNM - 650/2017,ACÓRDÃO DE CÂMARA,20,/2017/05/688448/133/2/117temp/ACOCAM.pdf,NaN,NaN,NaN,NaN,NaN,ACORDÃO Nº 1.269/2017\n\n\nPROCESSO TC Nº 0002...,"[{'page': 1, 'x0': 244, 'x1': 382, 'bottom': 1...",acordao,1269,2017.0,TC/000244/2017,TC/000244/2017


### Remover colunas que não serão aproveitadas

In [5]:
df_decisoes.columns

Index(['cod_tce', 'assunto', 'tipo_processo', 'cod_ato', 'ato', 'tipo_ato',
       'peca', 'caminho', 'conselheiro', 'municipio', 'ug', 'procurador',
       'ano_exercicio', 'texto', 'linhas', 'decisao_tipo', 'decisao_numero',
       'decisao_ano', 'cod_processo', 'cod_processo_aj'],
      dtype='object')

In [20]:
df_decisoes = df_decisoes[['cod_tce', 'cod_processo',  'tipo_processo', 'cod_ato', 'tipo_ato', 'peca', 'caminho', \
                           'conselheiro', 'texto', 'linhas', 'decisao_tipo', 'decisao_numero', 'decisao_ano']]

In [21]:
df_decisoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18921 entries, 0 to 18920
Data columns (total 13 columns):
cod_tce           18921 non-null object
cod_processo      18921 non-null object
tipo_processo     18921 non-null object
cod_ato           18921 non-null int64
tipo_ato          18921 non-null object
peca              18921 non-null int64
caminho           18921 non-null object
conselheiro       15713 non-null object
texto             18877 non-null object
linhas            18918 non-null object
decisao_tipo      18866 non-null object
decisao_numero    18866 non-null object
decisao_ano       18866 non-null float64
dtypes: float64(1), int64(2), object(10)
memory usage: 1.9+ MB


In [22]:
df_decisoes.head()

,cod_tce,cod_processo,tipo_processo,cod_ato,tipo_ato,peca,caminho,conselheiro,texto,linhas,decisao_tipo,decisao_numero,decisao_ano
0,000132/2017,TC/000132/2017,CONTROLE SOCIAL - DENÚNCIA,1284960,ACÓRDÃO DE CÂMARA,23,/2017/06/688336/133/2/805temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1647/2017\n\n\nPROCESSO TC Nº 00013...,"[{'page': 1, 'x0': 245, 'x1': 380, 'bottom': 1...",acordao,1647,2017.0
1,000189/2014,TC/000189/2014,INATIVAÇÃO - APOSENTADORIA,262333,ACÓRDÃO DE CÂMARA,10,/2014/07/623635/12/2/587temp/ACOCAM.pdf,NaN,ACÓRDÃO N.º 788/2014\n\nProcesso TC/000189/201...,"[{'page': 1, 'x0': 70, 'x1': 316, 'bottom': 13...",acordao,788,2014.0
2,000135/2020,TC/000135/2020,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,2462530,ACÓRDÃO DE CÂMARA,15,/2020/03//770683/8/2/17032020_095623_46033325D...,NaN,ACÓRDÃO Nº 317/2020\nPROCESSO: TC/000135/2020\...,"[{'page': 1, 'x0': 245, 'x1': 377, 'bottom': 1...",acordao,317,2020.0
3,000271/2017,TC/000271/2017,FISCALIZAÇÃO - AUDITORIA,1389052,ACÓRDÃO DO PLENO,35,/2017/09/688475/133/3/604temp/ACOPLE.pdf,NaN,ACORDÃO Nº 2.504/17\n\nPROCESSO TC Nº 000271/2...,"[{'page': 1, 'x0': 250, 'x1': 376, 'bottom': 1...",acordao,2504,2017.0
4,000244/2017,TC/000244/2017,CONTROLE SOCIAL - DENÚNCIA,1255656,ACÓRDÃO DE CÂMARA,20,/2017/05/688448/133/2/117temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1.269/2017\n\n\nPROCESSO TC Nº 0002...,"[{'page': 1, 'x0': 244, 'x1': 382, 'bottom': 1...",acordao,1269,2017.0


In [23]:
df_decisoes.loc[0]['linhas']

"[{'page': 1, 'x0': 245, 'x1': 380, 'bottom': 145, 'text': 'ACORDÃO Nº 1647/2017 '}, {'page': 1, 'x0': 85, 'x1': 87, 'bottom': 163, 'text': ' '}, {'page': 1, 'x0': 85, 'x1': 88, 'bottom': 177, 'text': ' '}, {'page': 1, 'x0': 85, 'x1': 254, 'bottom': 198, 'text': 'PROCESSO TC Nº 000132/2017 '}, {'page': 1, 'x0': 85, 'x1': 194, 'bottom': 219, 'text': 'DECISÃO Nº 343/17 '}, {'page': 1, 'x0': 85, 'x1': 567, 'bottom': 239, 'text': 'ASSUNTO: DENÚNCIA CONTRA P. M. DE CALDEIRÃO GRANDE DO PIAUÍ, EXERCÍCIO '}, {'page': 1, 'x0': 85, 'x1': 567, 'bottom': 260, 'text': 'FINANCEIRO DE 2016. SUPOSTAS IRREGULARIDADES NO PREGÃO PRESENCIAL Nº '}, {'page': 1, 'x0': 85, 'x1': 567, 'bottom': 281, 'text': '031/2017, TENDO COMO OBJETO A AQUISIÇÃO DE MEDICAMENTOS E MATERIAL '}, {'page': 1, 'x0': 85, 'x1': 508, 'bottom': 301, 'text': 'HOSPITALAR, A QUAL NÃO TERIA SIDO CADASTRADA NO SITE DO TCE/PI. '}, {'page': 1, 'x0': 85, 'x1': 518, 'bottom': 322, 'text': 'DENUNCIANTE: TRIBUNAL DE CONTAS DO ESTADO DO PIAUÍ – T

In [24]:
linhas_teste = eval(df_decisoes.loc[0]['linhas'])
type(linhas_teste)

list

In [25]:
linhas_teste

[{'page': 1,
  'x0': 245,
  'x1': 380,
  'bottom': 145,
  'text': 'ACORDÃO Nº 1647/2017 '},
 {'page': 1, 'x0': 85, 'x1': 87, 'bottom': 163, 'text': ' '},
 {'page': 1, 'x0': 85, 'x1': 88, 'bottom': 177, 'text': ' '},
 {'page': 1,
  'x0': 85,
  'x1': 254,
  'bottom': 198,
  'text': 'PROCESSO TC Nº 000132/2017 '},
 {'page': 1, 'x0': 85, 'x1': 194, 'bottom': 219, 'text': 'DECISÃO Nº 343/17 '},
 {'page': 1,
  'x0': 85,
  'x1': 567,
  'bottom': 239,
  'text': 'ASSUNTO: DENÚNCIA CONTRA P. M. DE CALDEIRÃO GRANDE DO PIAUÍ, EXERCÍCIO '},
 {'page': 1,
  'x0': 85,
  'x1': 567,
  'bottom': 260,
  'text': 'FINANCEIRO DE 2016. SUPOSTAS IRREGULARIDADES NO PREGÃO PRESENCIAL Nº '},
 {'page': 1,
  'x0': 85,
  'x1': 567,
  'bottom': 281,
  'text': '031/2017, TENDO COMO OBJETO A AQUISIÇÃO DE MEDICAMENTOS E MATERIAL '},
 {'page': 1,
  'x0': 85,
  'x1': 508,
  'bottom': 301,
  'text': 'HOSPITALAR, A QUAL NÃO TERIA SIDO CADASTRADA NO SITE DO TCE/PI. '},
 {'page': 1,
  'x0': 85,
  'x1': 518,
  'bottom': 322,
 

### Extrair os grupos

In [26]:
def extrai_grupos_acordao(linhas):
    if len(linhas) == 0: return None    
    
    x0_ementa = 210
    
    if(isinstance(linhas, str)):
        linhas = eval(linhas)
        
    def muda_parte(qtde=1):
        for i in range(qtde):
            parte = partes_acordao.pop(0)
        return parte, ''
    
    def eh_vazio(string):
        return not string.strip()
    
    def eh_bloco_ementa(text, x0):
        #return int(x0) > (int(page.width)/2)  # Deveria ser assim (depois da metade, mas 026455/2017 (Cons. Waltânia) não está)
        nonlocal x0_ementa
        if text.startswith('ementa'):
            x0_ementa = int(x0) - 10 # (pequeno ajuste)
            
        return int(x0) >= x0_ementa
    
    partes_acordao = ['Acordao', 'info_processuais', 'ementa', 'sumario', 'disposicoes']
    dados = dict()
    cabecalho = []
    pagina_linha_anteior = 0
    
    parte, texto_parte = muda_parte()
    testar_se_cabecalho = False

    for linha in linhas:
        if (pagina_linha_anteior != linha['page'] and linha['page'] > 1):
            testar_se_cabecalho = True

        text = clean_text(linha['text'])
        if testar_se_cabecalho:
            if text in cabecalho: 
                continue
            else: testar_se_cabecalho = False

        if parte == 'Acordao':
            if (text.startswith('acordao') or text.startswith('parecer previo')):
                dados[parte] = linha['text'].strip()
                parte, texto_parte = muda_parte()
            else:
                cabecalho.append(text)

            continue
        elif parte == 'info_processuais':
            if eh_vazio(text): continue
            if not eh_bloco_ementa(text, linha['x0']):
                texto_parte = texto_parte + linha['text'].strip() + '\n'
            else:
                dados[parte] = texto_parte.strip()
                parte, texto_parte = muda_parte()
                texto_parte = texto_parte + linha['text'].strip() + '\n'
                continue
        elif parte == 'ementa':
            if eh_bloco_ementa(text, linha['x0']) and not text.startswith('sumario'):
                texto_parte = texto_parte + linha['text'].strip() + '\n'
            else:
                dados[parte] = texto_parte.strip()
                qtde = 1 if text.startswith('sumario') else 2
                parte, texto_parte = muda_parte(qtde)
                texto_parte = texto_parte + linha['text'].strip() + '\n'
                continue
        elif parte == 'sumario':
            if eh_bloco_ementa(text, linha['x0']):
                texto_parte = texto_parte + linha['text'].strip() + '\n'
            else:
                dados[parte] = texto_parte.strip()
                parte, texto_parte = muda_parte()
                texto_parte = texto_parte + linha['text'].strip() + '\n'
                continue
        elif parte == 'disposicoes':
            texto_parte = texto_parte + linha['text'].strip() + '\n'
        
        pagina_linha_anteior = linha['page']

    dados[parte] = texto_parte.strip()       
    
    return dados


In [27]:
acordao = extrai_grupos_acordao(df_decisoes.loc[2]['linhas'])
acordao

{'Acordao': 'ACÓRDÃO Nº 317/2020',
 'info_processuais': 'PROCESSO: TC/000135/2020\nASSUNTO: RECURSO DE RECONSIDERAÇÃO EM FACE DO ACÓRDÃO Nº\n1.912/2019 (PRESTAÇÃO DE CONTAS DA CÂMARA MUNICIPAL\nDE NAZÁRIA – TC/005130/2015)\nÓRGÃO: CÂMARA MUNICIPAL DE NAZÁRIA – EXERCÍCIO 2015\nRECORRENTE: MARCELLO SOARES BEZERRA FONSECA\nRELATORA: CONSª. WALTÂNIA MARIA N. DE SOUSA LEAL ALVARENGA\nPROCURADOR: JOSÉ ARAÚJO PINHEIRO JÚNIOR\nADVOGADA: NAIARA DE MORAES E SILVA – OAB/PI Nº 5127',
 'ementa': 'EMENTA: RECURSO DE RECONSIDERAÇÃO. FALHAS QUE\nNÃO ENSEJAM A IRREGULARIDADE DAS CONTAS.\nPROVIMENTO DO RECURSO.\nNa hipótese de o recorrente demonstrar, em sede recursal, que as\nfalhas não são de gravidade suficiente para julgar irregulares tais\ncontas, o Acórdão recorrido é passível de modificação.',
 'sumario': 'Sumário. Recurso de Reconsideração em face do Acórdão nº\n1.912/2019, referente à Câmara Municipal de NAZÁRIA – Exercício\n2015. Preenchimento dos pressupostos de Admissibilidade.\nConhecimento

### Criar campos no dataset com base nos grupos

In [28]:
#df_decisoes['Acordao'] = ''
df_decisoes['info_processuais'] = ''
df_decisoes['ementa'] = ''
df_decisoes['sumario'] = ''

total_registros = len(df_decisoes)
for ind in df_decisoes.index: 

    if (isinstance(df_decisoes.loc[ind]['linhas'], str)):
        print(f'Extraindo dados: {ind} de {total_registros-1}')
        acordao = extrai_grupos_acordao(df_decisoes.loc[ind]['linhas'])    
        #df_decisoes.loc[ind, ['Acordao']] = acordao.get('Acordao')
        df_decisoes.loc[ind, ['info_processuais']] = acordao.get('info_processuais')
        df_decisoes.loc[ind, ['ementa']] = acordao.get('ementa')
        df_decisoes.loc[ind, ['sumario']] = acordao.get('sumario')


Extraindo dados: 0 de 18920
Extraindo dados: 1 de 18920
Extraindo dados: 2 de 18920
Extraindo dados: 3 de 18920
Extraindo dados: 4 de 18920
Extraindo dados: 5 de 18920
Extraindo dados: 6 de 18920
Extraindo dados: 7 de 18920
Extraindo dados: 8 de 18920
Extraindo dados: 9 de 18920
Extraindo dados: 10 de 18920
Extraindo dados: 11 de 18920
Extraindo dados: 12 de 18920
Extraindo dados: 13 de 18920
Extraindo dados: 14 de 18920
Extraindo dados: 15 de 18920
Extraindo dados: 16 de 18920
Extraindo dados: 17 de 18920
Extraindo dados: 18 de 18920
Extraindo dados: 19 de 18920
Extraindo dados: 20 de 18920
Extraindo dados: 21 de 18920
Extraindo dados: 22 de 18920
Extraindo dados: 23 de 18920
Extraindo dados: 24 de 18920
Extraindo dados: 25 de 18920
Extraindo dados: 26 de 18920
Extraindo dados: 27 de 18920
Extraindo dados: 28 de 18920
Extraindo dados: 29 de 18920
Extraindo dados: 30 de 18920
Extraindo dados: 31 de 18920
Extraindo dados: 32 de 18920
Extraindo dados: 33 de 18920
Extraindo dados: 34 de 1

Extraindo dados: 286 de 18920
Extraindo dados: 287 de 18920
Extraindo dados: 288 de 18920
Extraindo dados: 289 de 18920
Extraindo dados: 290 de 18920
Extraindo dados: 291 de 18920
Extraindo dados: 292 de 18920
Extraindo dados: 293 de 18920
Extraindo dados: 294 de 18920
Extraindo dados: 295 de 18920
Extraindo dados: 296 de 18920
Extraindo dados: 297 de 18920
Extraindo dados: 298 de 18920
Extraindo dados: 299 de 18920
Extraindo dados: 300 de 18920
Extraindo dados: 301 de 18920
Extraindo dados: 302 de 18920
Extraindo dados: 303 de 18920
Extraindo dados: 304 de 18920
Extraindo dados: 305 de 18920
Extraindo dados: 306 de 18920
Extraindo dados: 307 de 18920
Extraindo dados: 308 de 18920
Extraindo dados: 309 de 18920
Extraindo dados: 310 de 18920
Extraindo dados: 311 de 18920
Extraindo dados: 312 de 18920
Extraindo dados: 313 de 18920
Extraindo dados: 314 de 18920
Extraindo dados: 315 de 18920
Extraindo dados: 316 de 18920
Extraindo dados: 317 de 18920
Extraindo dados: 318 de 18920
Extraindo 

Extraindo dados: 566 de 18920
Extraindo dados: 567 de 18920
Extraindo dados: 568 de 18920
Extraindo dados: 569 de 18920
Extraindo dados: 570 de 18920
Extraindo dados: 571 de 18920
Extraindo dados: 572 de 18920
Extraindo dados: 573 de 18920
Extraindo dados: 574 de 18920
Extraindo dados: 575 de 18920
Extraindo dados: 576 de 18920
Extraindo dados: 577 de 18920
Extraindo dados: 578 de 18920
Extraindo dados: 579 de 18920
Extraindo dados: 580 de 18920
Extraindo dados: 581 de 18920
Extraindo dados: 582 de 18920
Extraindo dados: 583 de 18920
Extraindo dados: 584 de 18920
Extraindo dados: 585 de 18920
Extraindo dados: 586 de 18920
Extraindo dados: 587 de 18920
Extraindo dados: 588 de 18920
Extraindo dados: 589 de 18920
Extraindo dados: 590 de 18920
Extraindo dados: 591 de 18920
Extraindo dados: 592 de 18920
Extraindo dados: 593 de 18920
Extraindo dados: 594 de 18920
Extraindo dados: 595 de 18920
Extraindo dados: 596 de 18920
Extraindo dados: 597 de 18920
Extraindo dados: 598 de 18920
Extraindo 

Extraindo dados: 857 de 18920
Extraindo dados: 858 de 18920
Extraindo dados: 859 de 18920
Extraindo dados: 860 de 18920
Extraindo dados: 861 de 18920
Extraindo dados: 862 de 18920
Extraindo dados: 863 de 18920
Extraindo dados: 864 de 18920
Extraindo dados: 865 de 18920
Extraindo dados: 866 de 18920
Extraindo dados: 867 de 18920
Extraindo dados: 868 de 18920
Extraindo dados: 869 de 18920
Extraindo dados: 870 de 18920
Extraindo dados: 871 de 18920
Extraindo dados: 872 de 18920
Extraindo dados: 873 de 18920
Extraindo dados: 874 de 18920
Extraindo dados: 875 de 18920
Extraindo dados: 876 de 18920
Extraindo dados: 877 de 18920
Extraindo dados: 878 de 18920
Extraindo dados: 879 de 18920
Extraindo dados: 880 de 18920
Extraindo dados: 881 de 18920
Extraindo dados: 882 de 18920
Extraindo dados: 883 de 18920
Extraindo dados: 884 de 18920
Extraindo dados: 885 de 18920
Extraindo dados: 886 de 18920
Extraindo dados: 887 de 18920
Extraindo dados: 888 de 18920
Extraindo dados: 889 de 18920
Extraindo 

Extraindo dados: 1150 de 18920
Extraindo dados: 1151 de 18920
Extraindo dados: 1152 de 18920
Extraindo dados: 1153 de 18920
Extraindo dados: 1154 de 18920
Extraindo dados: 1155 de 18920
Extraindo dados: 1156 de 18920
Extraindo dados: 1157 de 18920
Extraindo dados: 1158 de 18920
Extraindo dados: 1159 de 18920
Extraindo dados: 1160 de 18920
Extraindo dados: 1161 de 18920
Extraindo dados: 1162 de 18920
Extraindo dados: 1163 de 18920
Extraindo dados: 1164 de 18920
Extraindo dados: 1165 de 18920
Extraindo dados: 1166 de 18920
Extraindo dados: 1167 de 18920
Extraindo dados: 1168 de 18920
Extraindo dados: 1169 de 18920
Extraindo dados: 1170 de 18920
Extraindo dados: 1171 de 18920
Extraindo dados: 1172 de 18920
Extraindo dados: 1173 de 18920
Extraindo dados: 1174 de 18920
Extraindo dados: 1175 de 18920
Extraindo dados: 1176 de 18920
Extraindo dados: 1177 de 18920
Extraindo dados: 1178 de 18920
Extraindo dados: 1179 de 18920
Extraindo dados: 1180 de 18920
Extraindo dados: 1181 de 18920
Extraind

...

Extraindo dados: 17580 de 18920
Extraindo dados: 17581 de 18920
Extraindo dados: 17582 de 18920
Extraindo dados: 17583 de 18920
Extraindo dados: 17584 de 18920
Extraindo dados: 17585 de 18920
Extraindo dados: 17586 de 18920
Extraindo dados: 17587 de 18920
Extraindo dados: 17588 de 18920
Extraindo dados: 17589 de 18920
Extraindo dados: 17590 de 18920
Extraindo dados: 17591 de 18920
Extraindo dados: 17592 de 18920
Extraindo dados: 17593 de 18920
Extraindo dados: 17594 de 18920
Extraindo dados: 17595 de 18920
Extraindo dados: 17596 de 18920
Extraindo dados: 17597 de 18920
Extraindo dados: 17598 de 18920
Extraindo dados: 17599 de 18920
Extraindo dados: 17600 de 18920
Extraindo dados: 17601 de 18920
Extraindo dados: 17602 de 18920
Extraindo dados: 17603 de 18920
Extraindo dados: 17604 de 18920
Extraindo dados: 17605 de 18920
Extraindo dados: 17606 de 18920
Extraindo dados: 17607 de 18920
Extraindo dados: 17608 de 18920
Extraindo dados: 17609 de 18920
Extraindo dados: 17610 de 18920
Extraind

Extraindo dados: 17852 de 18920
Extraindo dados: 17853 de 18920
Extraindo dados: 17854 de 18920
Extraindo dados: 17855 de 18920
Extraindo dados: 17856 de 18920
Extraindo dados: 17857 de 18920
Extraindo dados: 17858 de 18920
Extraindo dados: 17859 de 18920
Extraindo dados: 17860 de 18920
Extraindo dados: 17861 de 18920
Extraindo dados: 17862 de 18920
Extraindo dados: 17863 de 18920
Extraindo dados: 17864 de 18920
Extraindo dados: 17865 de 18920
Extraindo dados: 17866 de 18920
Extraindo dados: 17867 de 18920
Extraindo dados: 17868 de 18920
Extraindo dados: 17869 de 18920
Extraindo dados: 17870 de 18920
Extraindo dados: 17871 de 18920
Extraindo dados: 17872 de 18920
Extraindo dados: 17873 de 18920
Extraindo dados: 17874 de 18920
Extraindo dados: 17875 de 18920
Extraindo dados: 17876 de 18920
Extraindo dados: 17877 de 18920
Extraindo dados: 17878 de 18920
Extraindo dados: 17879 de 18920
Extraindo dados: 17880 de 18920
Extraindo dados: 17881 de 18920
Extraindo dados: 17882 de 18920
Extraind

Extraindo dados: 18121 de 18920
Extraindo dados: 18122 de 18920
Extraindo dados: 18123 de 18920
Extraindo dados: 18124 de 18920
Extraindo dados: 18125 de 18920
Extraindo dados: 18126 de 18920
Extraindo dados: 18127 de 18920
Extraindo dados: 18128 de 18920
Extraindo dados: 18129 de 18920
Extraindo dados: 18130 de 18920
Extraindo dados: 18131 de 18920
Extraindo dados: 18132 de 18920
Extraindo dados: 18133 de 18920
Extraindo dados: 18134 de 18920
Extraindo dados: 18135 de 18920
Extraindo dados: 18136 de 18920
Extraindo dados: 18137 de 18920
Extraindo dados: 18138 de 18920
Extraindo dados: 18139 de 18920
Extraindo dados: 18140 de 18920
Extraindo dados: 18141 de 18920
Extraindo dados: 18142 de 18920
Extraindo dados: 18143 de 18920
Extraindo dados: 18144 de 18920
Extraindo dados: 18145 de 18920
Extraindo dados: 18146 de 18920
Extraindo dados: 18147 de 18920
Extraindo dados: 18148 de 18920
Extraindo dados: 18149 de 18920
Extraindo dados: 18150 de 18920
Extraindo dados: 18151 de 18920
Extraind

Extraindo dados: 18391 de 18920
Extraindo dados: 18392 de 18920
Extraindo dados: 18393 de 18920
Extraindo dados: 18394 de 18920
Extraindo dados: 18395 de 18920
Extraindo dados: 18396 de 18920
Extraindo dados: 18397 de 18920
Extraindo dados: 18398 de 18920
Extraindo dados: 18399 de 18920
Extraindo dados: 18400 de 18920
Extraindo dados: 18401 de 18920
Extraindo dados: 18402 de 18920
Extraindo dados: 18403 de 18920
Extraindo dados: 18404 de 18920
Extraindo dados: 18405 de 18920
Extraindo dados: 18406 de 18920
Extraindo dados: 18407 de 18920
Extraindo dados: 18408 de 18920
Extraindo dados: 18409 de 18920
Extraindo dados: 18410 de 18920
Extraindo dados: 18411 de 18920
Extraindo dados: 18412 de 18920
Extraindo dados: 18413 de 18920
Extraindo dados: 18414 de 18920
Extraindo dados: 18415 de 18920
Extraindo dados: 18416 de 18920
Extraindo dados: 18417 de 18920
Extraindo dados: 18418 de 18920
Extraindo dados: 18419 de 18920
Extraindo dados: 18420 de 18920
Extraindo dados: 18421 de 18920
Extraind

Extraindo dados: 18661 de 18920
Extraindo dados: 18662 de 18920
Extraindo dados: 18663 de 18920
Extraindo dados: 18664 de 18920
Extraindo dados: 18665 de 18920
Extraindo dados: 18666 de 18920
Extraindo dados: 18667 de 18920
Extraindo dados: 18668 de 18920
Extraindo dados: 18669 de 18920
Extraindo dados: 18670 de 18920
Extraindo dados: 18671 de 18920
Extraindo dados: 18672 de 18920
Extraindo dados: 18673 de 18920
Extraindo dados: 18674 de 18920
Extraindo dados: 18675 de 18920
Extraindo dados: 18676 de 18920
Extraindo dados: 18677 de 18920
Extraindo dados: 18678 de 18920
Extraindo dados: 18679 de 18920
Extraindo dados: 18680 de 18920
Extraindo dados: 18681 de 18920
Extraindo dados: 18682 de 18920
Extraindo dados: 18683 de 18920
Extraindo dados: 18684 de 18920
Extraindo dados: 18685 de 18920
Extraindo dados: 18686 de 18920
Extraindo dados: 18687 de 18920
Extraindo dados: 18688 de 18920
Extraindo dados: 18689 de 18920
Extraindo dados: 18690 de 18920
Extraindo dados: 18691 de 18920
Extraind

In [29]:
#type(df_decisoes.loc[0]['linhas'])
isinstance(df_decisoes.loc[0]['linhas'], str)
acordao = extrai_grupos_acordao(df_decisoes.loc[0]['linhas'])  
acordao

{'Acordao': 'ACORDÃO Nº 1647/2017',
 'info_processuais': 'PROCESSO TC Nº 000132/2017\nDECISÃO Nº 343/17\nASSUNTO: DENÚNCIA CONTRA P. M. DE CALDEIRÃO GRANDE DO PIAUÍ, EXERCÍCIO\nFINANCEIRO DE 2016. SUPOSTAS IRREGULARIDADES NO PREGÃO PRESENCIAL Nº\n031/2017, TENDO COMO OBJETO A AQUISIÇÃO DE MEDICAMENTOS E MATERIAL\nHOSPITALAR, A QUAL NÃO TERIA SIDO CADASTRADA NO SITE DO TCE/PI.\nDENUNCIANTE: TRIBUNAL DE CONTAS DO ESTADO DO PIAUÍ – TCE/PI (VIA\nOUVIDORIA)\nDENUNCIADO: JOÃO VIANNEY DE SOUSA ALENCAR (PREFEITO)\nADVOGADOS: LUÍS VITOR SOUSA SANTOS – OAB Nº 12002.(PELO DENUNCIADO)\nPROCURADOR: LEANDRO MACIEL DO NASCIMENTO.\nRELATORA: LILIAN DE ALMEIDA VELOSO NUNES MARTINS.',
 'ementa': 'Denúncia contra a Prefeitura Municipal de\nCaldeirão Grande do Piauí. Unânime, divergindo\ndo MPC pela improcedência.',
 'disposicoes': 'Vistos, relatados e discutidos os presentes autos, considerando a informação do contraditório\nda III DFAM (Peça 15), o parecer do Ministério Público de Contas (Peças 17), con

In [30]:
#Sem ementa: 736, 744, 4388, 
# Analisar 117
indice = 6131
print(f'Processo: {df_decisoes.loc[indice]["cod_tce"]} - peça {df_decisoes.loc[indice]["peca"]}')
acordao = extrai_grupos_acordao(df_decisoes.loc[indice]['linhas'])
acordao
#acordao.get('sumario')
#df_decisoes.loc[indice]['linhas']
#type(em)

Processo: 013024/2014 - peça 50


{'Acordao': 'ACÓRDÃO Nº. 1.677/15',
 'info_processuais': '',
 'ementa': 'Representação. Estado do Piauí. Secretaria de\nAdministração. Exercício Financeiro de 2014.  Análise\ntécnica circunstanciada. Conhecimento e Procedência\nParcial da presente Representação. Revogação da Medida\nCautelar. Exclusão do responsável Raimundo Rodrigues\nAlves do presente processo de representação. Adoção de\nprocedimento próprio de fiscalização. Cumprimento de\ntodas as recomendações e determinações sugeridas pela\nDALC. Apensamento ao processo de prestação de contas\nda SEAD.',
 'disposicoes': 'PROCESSO: TC nº. 013.024/14 - Representação\nREPRESENTANTE: J. Nerval de Sousa - EPP\nREPRESENTADO: Secretaria de Administração do Estado do Piauí\nRELATOR: Conselheiro-Substituto Alisson Felipe de Araújo\nPROCURADOR: Márcio André Madeira de Vasconcelos\nADVOGADO: Dr. Vítor Tabatinga do Rêgo Lopes - OAB/PI nº. 6.989\n\nVistos, relatados e discutidos os presentes autos, decidiu o Plenário, unânime, em concordânci

In [31]:
df_decisoes.head()

,cod_tce,cod_processo,tipo_processo,cod_ato,tipo_ato,peca,caminho,conselheiro,texto,linhas,decisao_tipo,decisao_numero,decisao_ano,info_processuais,ementa,sumario
0,000132/2017,TC/000132/2017,CONTROLE SOCIAL - DENÚNCIA,1284960,ACÓRDÃO DE CÂMARA,23,/2017/06/688336/133/2/805temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1647/2017\n\n\nPROCESSO TC Nº 00013...,"[{'page': 1, 'x0': 245, 'x1': 380, 'bottom': 1...",acordao,1647,2017.0,PROCESSO TC Nº 000132/2017\nDECISÃO Nº 343/17\...,Denúncia contra a Prefeitura Municipal de\nCal...,None
1,000189/2014,TC/000189/2014,INATIVAÇÃO - APOSENTADORIA,262333,ACÓRDÃO DE CÂMARA,10,/2014/07/623635/12/2/587temp/ACOCAM.pdf,NaN,ACÓRDÃO N.º 788/2014\n\nProcesso TC/000189/201...,"[{'page': 1, 'x0': 70, 'x1': 316, 'bottom': 13...",acordao,788,2014.0,Processo TC/000189/2014\nAssunto: Aposentadori...,EMENTA: APOSENTADORIA VOLUNTÁRIA POR\nIDADE E ...,None
2,000135/2020,TC/000135/2020,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,2462530,ACÓRDÃO DE CÂMARA,15,/2020/03//770683/8/2/17032020_095623_46033325D...,NaN,ACÓRDÃO Nº 317/2020\nPROCESSO: TC/000135/2020\...,"[{'page': 1, 'x0': 245, 'x1': 377, 'bottom': 1...",acordao,317,2020.0,PROCESSO: TC/000135/2020\nASSUNTO: RECURSO DE ...,EMENTA: RECURSO DE RECONSIDERAÇÃO. FALHAS QUE\...,Sumário. Recurso de Reconsideração em face do ...
3,000271/2017,TC/000271/2017,FISCALIZAÇÃO - AUDITORIA,1389052,ACÓRDÃO DO PLENO,35,/2017/09/688475/133/3/604temp/ACOPLE.pdf,NaN,ACORDÃO Nº 2.504/17\n\nPROCESSO TC Nº 000271/2...,"[{'page': 1, 'x0': 250, 'x1': 376, 'bottom': 1...",acordao,2504,2017.0,PROCESSO TC Nº 000271/2017\nDECISÃO Nº 1.328/1...,EMENTA. INSPEÇÃO ORDINÁRIA. VERIFICAÇÃO DE\nRE...,None
4,000244/2017,TC/000244/2017,CONTROLE SOCIAL - DENÚNCIA,1255656,ACÓRDÃO DE CÂMARA,20,/2017/05/688448/133/2/117temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1.269/2017\n\n\nPROCESSO TC Nº 0002...,"[{'page': 1, 'x0': 244, 'x1': 382, 'bottom': 1...",acordao,1269,2017.0,PROCESSO TC Nº 000244/2017\nDECISÃO Nº 261/17\...,"Denúncia contra FUESPI. Unânime, concordando\n...",None


In [32]:
df_decisoes.tail()

,cod_tce,cod_processo,tipo_processo,cod_ato,tipo_ato,peca,caminho,conselheiro,texto,linhas,decisao_tipo,decisao_numero,decisao_ano,info_processuais,ementa,sumario
18916,020418/2019,TC/020418/2019,CONTROLE SOCIAL - DENÚNCIA,2599958,ACÓRDÃO DE CÂMARA,22,/2020/08//768949/13/2/31082020_210721_B84BEA01...,OLAVO REBELO DE CARVALHO FILHO,ACÓRDÃO Nº. 789/2020\nProcesso TC n° 020418/20...,"[{'page': 1, 'x0': 239, 'x1': 363, 'bottom': 1...",acordao,789,2020.0,Processo TC n° 020418/2019\nÓrgão de Deliberaç...,Denúncia formulada contra a Sra. Ana Célia da ...,None
18917,025790/2017,TC/025790/2017,CONTAS - TOMADA DE CONTAS ESPECIAL,2617099,ACÓRDÃO DE CÂMARA,49,/2020/09//713994/16/2/21092020_100626_30EA5B30...,JAYLSON FABIANH LOPES CAMPELO,ACÓRDÃO Nº 1.495/2020\nDECISÃO Nº 407/2020.\nP...,"[{'page': 1, 'x0': 214, 'x1': 371, 'bottom': 1...",acordao,1495,2020.0,DECISÃO Nº 407/2020.\nPROCESSO: TC/025790/2017...,EMENTA: PROCESSUAL. TOMADA DE CONTAS\nESPECIAL...,SUMÁRIO: TOMADA DE CONTAS ESPECIAL DA\nPREFEIT...
18918,016165/2019,TC/016165/2019,CONTROLE SOCIAL - REPRESENTAÇÃO,2631943,ACÓRDÃO DE CÂMARA,27,/2020/09//764696/120/2/29092020_102340_5914BF0...,DELANO CARNEIRO DA CUNHA CAMARA,ACÓRDÃO Nº 1.405/2020\nPROCESSO TC Nº 016165/2...,"[{'page': 1, 'x0': 244, 'x1': 379, 'bottom': 1...",acordao,1405,2020.0,PROCESSO TC Nº 016165/2019\nDECISÃO Nº 468/202...,EMENTA. PRESTAÇÃO DE CONTAS. PEDIDO\nDE BLOQUE...,Sumário. Representação. Câmara Municipal de\nP...
18919,018868/2018,TC/018868/2018,CONTROLE SOCIAL - REPRESENTAÇÃO,2595834,ACÓRDÃO,30,/2020/08//743068/119/75/27082020_154514_901DC6...,JACKSON NOBRE VERAS,ACÓRDÃO Nº 1.292/2020\nPROCESSO: TC/018868/201...,"[{'page': 1, 'x0': 254, 'x1': 361, 'bottom': 2...",acordao,1292,2020.0,PROCESSO: TC/018868/2018\nDECISÃO Nº 337/2020\...,EMENTA: PRESTAÇÃO DE CONTAS. BLOQUEIO\nDE CONT...,Sumário: Representação. Consórcio Intermunicip...
18920,019666/2019,TC/019666/2019,INATIVAÇÃO - APOSENTADORIA,2599773,ACÓRDÃO DE CÂMARA,10,/2020/08//768197/119/2/31082020_221212_009B1C1...,JACKSON NOBRE VERAS,ACÓRDÃO Nº 806/2020\nPROCESSO TC/019666/2019\n...,"[{'page': 1, 'x0': 258, 'x1': 357, 'bottom': 1...",acordao,806,2020.0,PROCESSO TC/019666/2019\nDECISÃO Nº 179/20\nAS...,EMENTA: APOSENTADORIA. INGRESSO DE SERVIDOR NO...,SUMÁRIO: APOSENTADORIA VOLUNTÁRIA POR IDADE E\...


In [34]:
df_decisoes['tamanho_ementa'] = df_decisoes[df_decisoes['ementa'].notna()]['ementa'].apply(len)

In [35]:
df_decisoes.tamanho_ementa.describe()

count    18864.000000
mean       304.045006
std        272.190107
min          0.000000
25%        160.000000
50%        238.000000
75%        371.000000
max       4515.000000
Name: tamanho_ementa, dtype: float64

In [36]:
df_decisoes[((df_decisoes.tamanho_ementa > 2000) & (df_decisoes.sumario.isna()))]

,cod_tce,cod_processo,tipo_processo,cod_ato,tipo_ato,peca,caminho,conselheiro,texto,linhas,decisao_tipo,decisao_numero,decisao_ano,info_processuais,ementa,sumario,tamanho_ementa
6015,011840/2014,TC/011840/2014,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,403364,ACÓRDÃO DO PLENO,13,/2014/12/635321/12/3/679temp/ACOPLE.pdf,NaN,ACÓRDÃO Nº 1.594/2014\nPROCESSO TC/011840/2014...,"[{'page': 1, 'x0': 85, 'x1': 319, 'bottom': 14...",acordao,1594,2014.0,PROCESSO TC/011840/2014.\nASSUNTO: RECURSO DE ...,EMENTA: RECURSO DE RECONSIDERAÇÃO. CONTAS DA\n...,None,4010.0
6022,011835/2014,TC/011835/2014,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,403369,ACÓRDÃO DO PLENO,12,/2014/12/635316/12/3/534temp/ACOPLE.pdf,NaN,ACÓRDÃO Nº 1.596/2014\nPROCESSO TC/011835/2014...,"[{'page': 1, 'x0': 85, 'x1': 319, 'bottom': 14...",acordao,1596,2014.0,PROCESSO TC/011835/2014.\nASSUNTO: RECURSO DE ...,EMENTA: RECURSO DE RECONSIDERAÇÃO. CONTAS DE\n...,None,3527.0
6106,014764/2014,TC/014764/2014,CONTAS - CONTAS DE GESTÃO,1097262,ACÓRDÃO DE CÂMARA,47,/2017/01/638245/16/2/968temp/ACOCAM.pdf,JAYLSON FABIANH LOPES CAMPELO,ACÓRDÃO Nº. 3348/2016\n\n\n\n\nDECISÃO Nº 625...,"[{'page': 1, 'x0': 186, 'x1': 404, 'bottom': 1...",acordao,3348,2016.0,DECISÃO Nº 625/16\nSESSÃO ORDINÁRIA DA PRIMEIR...,PRESTAÇÃO DE CONTAS DO HOSPITAL REGIONAL DE\nC...,None,2199.0
6109,014764/2014,TC/014764/2014,CONTAS - CONTAS DE GESTÃO,1090580,ACÓRDÃO DE CÂMARA,43,/2017/01/638245/16/2/67temp/ACOCAM.pdf,JAYLSON FABIANH LOPES CAMPELO,ACÓRDÃO Nº. 3348/2016\n\n\n\n\nDECISÃO Nº 625...,"[{'page': 1, 'x0': 186, 'x1': 404, 'bottom': 1...",acordao,3348,2016.0,DECISÃO Nº 625/16\nSESSÃO ORDINÁRIA DA PRIMEIR...,PRESTAÇÃO DE CONTAS DO HOSPITAL REGIONAL DE\nC...,None,2199.0
7079,014206/2014,TC/014206/2014,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,420362,ACÓRDÃO DO PLENO,14,/2015/02/637687/13/3/912temp/ACOPLE.pdf,NaN,ACÓRDÃO Nº. 308/2015\n\n\nRecurso de Reconside...,"[{'page': 1, 'x0': 233, 'x1': 393, 'bottom': 1...",acordao,308,2015.0,,Recurso de Reconsideração. Prestação de Contas...,None,2555.0
7342,014773/2014,TC/014773/2014,CONTAS - CONTAS DE GESTÃO,983806,ACÓRDÃO DE CÂMARA,28,/2016/09/638254/13/2/273temp/ACOCAM.pdf,OLAVO REBELO DE CARVALHO FILHO,ACÓRDÃO Nº. 2.363/2016\n\n\n\nPrestação de Con...,"[{'page': 1, 'x0': 227, 'x1': 400, 'bottom': 1...",acordao,2363,2016.0,,Prestação de Contas Anual do Laboratório Centr...,None,2382.0
8725,017872/2019,TC/017872/2019,DOS RECURSOS - EMBARGOS DE DECLARAÇÃO,2500021,ACÓRDÃO DO PLENO,21,/2020/05//766403/133/3/21052020_090250_4A03B73...,LILIAN DE ALMEIDA VELOSO NUNES MARTINS,ACORDÃO Nº 402/2020\nPROCESSO TC Nº 017872/201...,"[{'page': 1, 'x0': 248, 'x1': 374, 'bottom': 1...",acordao,402,2020.0,PROCESSO TC Nº 017872/2019\nDECISÃO Nº 317/20\...,EMENTA. EMBARGOS DE DECLARAÇÃO.\nPROPOSTA DE E...,None,2043.0
10317,006538/2013,TC/02728/2013,CONTAS - CONTAS DE GESTÃO,1087097,ACÓRDÃO DE CÂMARA,47,/2016/12/609110/13/2/143temp/ACOCAM.pdf,OLAVO REBELO DE CARVALHO FILHO,ACÓRDÃO Nº. 2.864/2016\n\n\nAuditoria “in loco...,"[{'page': 1, 'x0': 227, 'x1': 400, 'bottom': 1...",acordao,2864,2016.0,,Auditoria “in loco” realizada no Regime Própri...,None,3078.0
10908,006492/2013,TC/02682/2013,CONTAS - CONTAS DE GESTÃO,851359,PARECER PRÉVIO,63,/2016/05/609064/16/42/987temp/PARPRE.pdf,JAYLSON FABIANH LOPES CAMPELO,PARECER PRÉVIO Nº 106/2016\n\n\n\nDECISÃO Nº. ...,"[{'page': 1, 'x0': 197, 'x1': 400, 'bottom': 1...",parecer,106,2016.0,DECISÃO Nº. 273/2016\nSESSÃO ORDINÁRIA DA PRIM...,PRESTAÇÃO DE CONTAS DA PREFEITURA MUNICIPAL DE...,None,2215.0
14545,013048/2012,TC-E-013048/2012,CONTAS - CONTAS DE GESTÃO,389363,ACÓRDÃO DE CÂMARA,51,/2015/01/532509/12/2/432temp/ACOCAM.pdf,WALTANIA MARIA NOGUEIRA DE SOUSA LEAL ALVARENGA,ACÓRDÃO Nº 1.506/2014.\n\nEMENTA: PRESTAÇÃO DE...,"[{'page': 1, 'x0': 191, 'x1': 333, 'bottom': 1...",acordao,1506,2014.0,,EMENTA: PRESTAÇÃO DE CONTAS DO FUNDO DE MANUTE...,None,2909.0


### Remover ementas incorretas
Analisando os registros acima, percebi que algumas ementas estavam erradas e deveriam ser removidas
<p> Essas estão com x0 errado, fazendo com que a ementa fique com quase todo texto<br/>
[6015, 6022]

In [38]:
df_decisoes.loc[6015,['ementa']][0]

'EMENTA: RECURSO DE RECONSIDERAÇÃO. CONTAS DA\nCÂMARA MUNICIPAL DE RIBEIRA DO PIAUÍ, EXERCÍCIO 2011.\nPRESSUPOSTOS RECURSAIS PREENCHIDOS. CONHECIMENTO E\nPROVIMENTO. ARGUMENTOS PROFERIDOS SÃO EM PARTE\nPROCEDENTES. AS FALHAS DETECTADAS SÃO DE POUCA\nEXPRESSIVIDADE. PONDERAÇÕES APRESENTADAS PELA\nADVOGADA EM PLENÁRIO. PRINCÍPIOS DO FORMALISMO\nMODERADO E O DA BUSCA PELA VERDADE MATERIAL. NÃO\nHOUVE MALVERSAÇÃO DE RECURSOS PÚBLICOS OU DESVIO DE\nRECURSOS. MODIFICAÇÃO DO JULGAMENTO DE\nIRREGULARIDAE PARA REGULARIDADE COM RESSALVAS.\nMANUTENÇÃO DA MULTA VALOR EQUIVALENTE À 800 UFR/PI\nAO SR. RONIVALDO CAMPELO DO NASCIMENTO.\n\n\nVistos, relatados e discutidos os presentes autos, versando sobre Recurso de Reconsideração interposto pelo Sr.\nRonivaldo Campelo do Nascimento, gestor da Câmara Municipal de Ribeira do Piauí, exercício de 2011, por meio de sua\nadvogada, contra decisão deste Tribunal proferida nos autos do Processo TCE nº 15582/2012 (Prestação de Contas Anual da\nPrefeitura Munic

In [39]:
df_decisoes.loc[6015,['ementa']] = None
df_decisoes.loc[6022,['ementa']] = None

In [40]:
df_decisoes.loc[6015,['ementa']]

ementa    None
Name: 6015, dtype: object

## Exportar dados para csv

In [24]:
# Antes, remover coluna linhas para diminuir o tamanho do arquivo
#df_decisoes.drop(['linhas'], axis=1, inplace=True)

In [41]:
caminho = Path("../../data/raw/tce_acordaos_e_pareceres_com_ementas.csv")
df_decisoes.to_csv(caminho, index=False, sep=';')